---
jupyter: python3
---

In [ ]:
import pandas as pd
from tqdm import tqdm

from CryptoFraudDetection.utils.enums import LoggerMode
from CryptoFraudDetection.utils.logger import Logger
from CryptoFraudDetection.elasticsearch.data_retrieval import search_data
from CryptoFraudDetection.elasticsearch.data_insertion import insert_dataframe

In [ ]:
response = search_data(index="reddit_posts", q="*", size=6000)

In [ ]:
response["hits"]["hits"][0].keys()

In [ ]:
response["hits"]["hits"][0]["_source"]

In [ ]:
# Recursive function to flatten comments with parent_id
def flatten_comments(comments, parent_id):
    rows = []
    for comment in comments:
        # Extract comment details
        rows.append(
            {
                "id": comment["id"],
                "parent_id": parent_id,
                "author": comment["author"],
                "body": comment["body"],
                "created": comment["created"],
                "depth": comment["depth"],
                "edited": comment["edited"],
                "score": comment["score"],
                "search_query": comment["search_query"],
                "subreddit": comment["subreddit"],
            }
        )
        # If the comment has nested replies, process them recursively
        if comment.get("comments"):
            rows.extend(flatten_comments(comment["comments"], parent_id=comment["id"]))
    return rows


# Function to flatten the entire JSON structure
def flatten_json(json_data):
    # Extract submission data
    submission = {
        "id": json_data["id"],
        "parent_id": None,
        "author": json_data["author"],
        "body": json_data["body"],
        "created": json_data["created"],
        "depth": json_data["depth"],
        "edited": json_data["edited"],
        "score": json_data["score"],
        "search_query": json_data["search_query"],
        "subreddit": json_data["subreddit"],
        "title": json_data["title"],  # Specific to submission
        "url": json_data["url"],  # Specific to submission
        "num_comments": json_data["num_comments"],  # Specific to submission
    }

    # Flatten comments
    comments = flatten_comments(json_data["comments"], parent_id=json_data["id"])

    # Combine submission and comments into a single dataset
    all_data = [submission] + comments

    # Convert to DataFrame
    return pd.DataFrame(all_data)

In [ ]:
df = pd.DataFrame()
for post in tqdm(response["hits"]["hits"]):
    if df.empty:
        df = flatten_json(post["_source"])
    else:
        df = pd.concat([df, flatten_json(post["_source"])])

df = df.convert_dtypes()

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.to_parquet("../data/processed/reddit_posts.parquet", index=False)

In [ ]:
logger_ = Logger(name="reddit_unwrap_posts", level=LoggerMode.DEBUG, log_dir="../logs")

_ = insert_dataframe(logger=logger_, index="reddit_posts_unwrapped", df=df)